In [3]:
import matplotlib.pyplot as plt
import rasterio 
from rasterio.plot import plotting_extent, show
from rasterio.merge import merge
import geopandas as gpd
from osgeo import gdal
import numpy as np
from skimage import io


def data_fusion(ortho_nir, ortho_rgb, ortho_mspec, outputpath):
    
    ortho_NIR = ortho_nir
    ortho_rgb = ortho_rgb
    ortho_mspec = ortho_mspec
    
    g = gdal.Open(ortho_NIR)
    c = gdal.Open(ortho_mspec)
    x = gdal.Open(ortho_rgb)
    
    dsReprj_mspec= gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_mspec_2.tif", c, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_nir = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_nir_2.tif", g, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_rgb = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_rgb_2.tif", x, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    
    stack = []
    
    rgb_red = dsReprj_rgb.GetRasterBand(1).ReadAsArray().astype(np.float32)
    rgb_green = dsReprj_rgb.GetRasterBand(2).ReadAsArray().astype(np.float32)
    rgb_blue =  dsReprj_rgb.GetRasterBand(3).ReadAsArray().astype(np.float32)
    stack.append(rgb_red)
    stack.append(rgb_green)
    stack.append(rgb_blue)
    
    mspec_rededge = dsReprj_mspec.GetRasterBand(3).ReadAsArray().astype(np.float32) 
    stack.append(mspec_rededge)
   
    b3 = dsReprj_nir.GetRasterBand(2).ReadAsArray().astype(np.float32)
    b4 = dsReprj_nir.GetRasterBand(3).ReadAsArray().astype(np.float32)
    nir_ndvi = (b4 - b3)/(b4 + b3)
    nir_nir = dsReprj_nir.GetRasterBand(3).ReadAsArray().astype(np.float32)
    stack.append(nir_nir)
    stack.append(nir_ndvi)
    
    #print(stack)
    
    stack_data = np.array(stack, dtype=object)
    print(stack_data[1].shape)
    print(stack_data[1].shape[1])
    print(stack_data[1].shape[0])
    driver = gdal.GetDriverByName ( "GTiff" )
    DataSet = driver.Create(outputpath, stack_data[1].shape[1], stack_data[1].shape[0], len(stack_data), gdal.GDT_Float32)
    DataSet.SetGeoTransform(x.GetGeoTransform())
    DataSet.SetProjection(x.GetProjection())
    print('---------------------')
    for i, image in enumerate(stack_data, 1):
        DataSet.GetRasterBand(i).WriteArray(image)
        DataSet.GetRasterBand(i).SetNoDataValue(-9999.0)
    DataSet.FlushCache()

In [4]:
#ortho_NIR = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/nir.tif"
#ortho_rgb = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/rgb.tif"
#ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/mspec.tif"
#outputpath = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_channelFusion_TEST.tif"

ortho_NIR = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/nir_2.tif"
ortho_rgb = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/rgb_2.tif"
ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/mspec_2.tif"
outputpath = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_channelFusion_5.tif"

data_fusion(ortho_NIR, ortho_rgb, ortho_mspec, outputpath)

C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_2852/1734432870.py:39: RuntimeWarning: invalid value encountered in true_divide
  nir_ndvi = (b4 - b3)/(b4 + b3)


(5808, 6668)
6668
5808
---------------------
